In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.layers import *
from local.data.all import *
from local.notebook.showdoc import show_doc
from local.optimizer import *
from local.learner import *

In [ ]:
#default_exp callback.progress

# Progress and logging callbacks

> Callback and helper function to track progress of training or log results

In [ ]:
from local.utils.test import *

## ProgressCallback -

In [ ]:
# export
@docs
class ProgressCallback(Callback):
    "A `Callback` to handle the display of progress bars"
    run_after=Recorder

    def begin_fit(self):
        assert hasattr(self.learn, 'recorder')
        self.mbar = master_bar(list(range(self.n_epoch)))
        self.mbar.on_iter_begin()
        self.old_logger,self.learn.logger = self.logger,self._write_stats
        self._write_stats(self.recorder.metric_names)

    def begin_epoch(self):    self.mbar.update(self.epoch)
    def begin_train(self):    self._launch_pbar()
    def begin_validate(self): self._launch_pbar()
    def after_train(self):    self.pbar.on_iter_end()
    def after_validate(self): self.pbar.on_iter_end()
    def after_batch(self):
        self.pbar.update(self.iter+1)
        self.pbar.comment = f'{self.smooth_loss:.4f}'

    def _launch_pbar(self):
        self.pbar = progress_bar(self.dl, parent=self.mbar)
        self.pbar.update(0)

    def after_fit(self):
        self.mbar.on_iter_end()
        self.learn.logger = self.old_logger

    def _write_stats(self, log):
        self.mbar.write([f'{l:.6f}' if isinstance(l, float) else str(l) for l in log], table=True)

    _docs = dict(begin_fit="Setup the master bar over the epochs",
                 begin_epoch="Update the master bar",
                 begin_train="Launch a progress bar over the training dataloader",
                 begin_validate="Launch a progress bar over the validation dataloader",
                 after_train="Close the progress bar over the training dataloader",
                 after_validate="Close the progress bar over the validation dataloader",
                 after_batch="Update the current progress bar",
                 after_fit="Close the master bar")

defaults.callbacks = [TrainEvalCallback, Recorder, ProgressCallback]

In [ ]:
learn = synth_learner()
learn.fit(5)

In [ ]:
#hide
assert not learn.progress.mbar.child.is_active
lines = learn.progress.mbar.lines
test_eq(learn.recorder.metric_names, lines[0])
for i,(l,v) in enumerate(zip(lines[1:],learn.recorder.values)):
    test_eq(l[:-1], [str(i)] + [f'{x:.6f}' for x in v])

In [ ]:
show_doc(ProgressCallback.begin_fit)

In [ ]:
show_doc(ProgressCallback.begin_epoch)

In [ ]:
show_doc(ProgressCallback.begin_train)

In [ ]:
show_doc(ProgressCallback.begin_validate)

In [ ]:
show_doc(ProgressCallback.after_batch)

In [ ]:
show_doc(ProgressCallback.after_train)

In [ ]:
show_doc(ProgressCallback.after_validate)

In [ ]:
show_doc(ProgressCallback.after_fit)

## ShowGraphCallback -

In [ ]:
# export
class ShowGraphCallback(Callback):
    "Update a graph of training and validation loss"
    run_after=ProgressCallback

    def begin_fit(self):
        self.nb_batches = []
        assert hasattr(self.learn, 'progress')

    def after_train(self): self.nb_batches.append(self.train_iter)

    def after_epoch(self):
        "Plot validation loss in the pbar graph"
        rec = self.learn.recorder
        iters = range_of(rec.losses)
        val_losses = [v[1] for v in rec.values]
        x_bounds = (0, (self.n_epoch - len(self.nb_batches)) * self.nb_batches[0] + len(rec.losses))
        y_bounds = (0, max((max(Tensor(rec.losses)), max(Tensor(val_losses)))))
        self.progress.mbar.update_graph([(iters, rec.losses), (self.nb_batches, val_losses)], x_bounds, y_bounds)

In [ ]:
#slow
learn = synth_learner(cbs=ShowGraphCallback())
learn.fit(10)

## CSVLogger -

In [ ]:
# export
class CSVLogger(Callback):
    order=30 #Need to run after the recorder
    "Log the results displayed in `learn.path/fname`"
    def __init__(self, fname='history.csv', append=False):
        self.fname,self.append = Path(fname),append

    def read_log(self):
        "Convenience method to quickly access the log."
        return pd.read_csv(self.path/self.fname)

    def begin_fit(self):
        "Prepare file with metric names."
        self.path.parent.mkdir(parents=True, exist_ok=True)
        self.file = (self.path/self.fname).open('a' if self.append else 'w')
        self.file.write(','.join(self.recorder.metric_names) + '\n')
        self.old_logger,self.learn.logger = self.logger,self._write_line

    def _write_line(self, log):
        "Write a line with `log` and call the old logger."
        self.file.write(','.join([str(t) for t in log]) + '\n')
        self.old_logger(log)

    def after_fit(self):
        "Close the file and clean up."
        self.file.close()
        self.learn.logger = self.old_logger

The results are appened to an existing file if `append`, or they overwrite it otherwise.

In [ ]:
learn = synth_learner(cbs=CSVLogger())
learn.fit(5)

In [ ]:
show_doc(CSVLogger.read_log)

In [ ]:
df = learn.csv_logger.read_log()
test_eq(df.columns.values, learn.recorder.metric_names)
for i,v in enumerate(learn.recorder.values):
    test_close(df.iloc[i][:3], [i] + v)
os.remove(learn.path/learn.csv_logger.fname)

In [ ]:
show_doc(CSVLogger.begin_fit)

In [ ]:
show_doc(CSVLogger.after_fit)

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)